<a href="https://colab.research.google.com/github/Djude1/OpenAIDevice_For_VisualImpairment/blob/main/yolo_wandb_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚶 台灣視障輔助系統 - YOLO 訓練筆記本

這個筆記本用於訓練適合台灣環境的 YOLO 模型，包含：
- 人行道邊緣檢測
- 斑馬線識別
- 障礙物檢測（機車、電線桿等）

## 📋 使用前準備

1. **WandB 帳號**: 註冊 https://wandb.ai
2. **數據集**: 確保數據已上傳到 Google Drive 或 GitHub
3. **GPU**: 建議使用 Colab Pro (T4/V100)

## 👥 組員資訊

修改下面的 `EXPERIMENT_NAME` 以區分不同組員的實驗：
- `member1_sidewalk_v1`
- `member2_crosswalk_v1`
- `member3_obstacle_v1`
- `member4_integration_v1`

---
## 🔧 第一步：環境設置

In [ ]:
# 檢查 GPU 是否可用
!nvidia-smi

# 安裝必要套件
!pip install -q ultralytics wandb albumentations pyyaml

import os
import yaml
from pathlib import Path
print("✅ 套件安裝完成！")

Tue Jan 13 13:34:51 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

---
## 📂 第二步：連接資料來源

選擇以下其中一種方式：
1. **Google Drive** - 如果數據在你的雲端硬碟
2. **GitHub** - 如果數據在專案倉庫
3. **直接下載** - 從其他來源下載

In [ ]:
# === 方法 1: 掛載 Google Drive ===     //只負責照片
from google.colab import drive
drive.mount('/content/drive')

# 設定數據路徑（修改成你的實際路徑）
DRIVE_DATA_PATH = "/content/drive/MyDrive/taiwan_blind_navigation/dataset"
print(f"✅ Google Drive 已掛載！數據路徑: {DRIVE_DATA_PATH}")

Mounted at /content/drive
✅ Google Drive 已掛載！數據路徑: /content/drive/MyDrive/taiwan_blind_navigation/dataset


In [ ]:
# === 方法 2: Clone GitHub 專案 ===   //只負責code
!git clone https://github.com/Djude1/OpenAIDevice_For_VisualImpairment.git
%cd OpenAIDevice_For_VisualImpairment

# 拉取最新代碼
!git pull origin main
print("✅ GitHub 專案已同步！")

---
## 🔑 第三步：WandB 設定

In [ ]:
import wandb

# WandB 登入（會跳出連結，複製 API Key）
wandb.login()

# 或者直接貼上你的 API Key
# wandb.login(key="你的API金鑰")

print("✅ WandB 登入成功！")

---
## ⚙️ 第四步：訓練配置

### 重要參數說明：
- `EXPERIMENT_NAME`: 實驗名稱，**每個組員要改成不同的名字**
- `MODEL_SIZE`: 模型大小 (n/s/m/l/x)     //m 寫死
- `EPOCHS`: 訓練輪數
- `BATCH_SIZE`: 批次大小（顯存不足可降低）
- `IMAGE_SIZE`: 輸入圖片大小

In [ ]:
# ========== 核心配置（組員需修改這裡）==========
EXPERIMENT_NAME = "member1_sidewalk_v1"  # 🔴 組員修改這裡！
WANDB_PROJECT = "taiwan-blind-navigation"  # WandB 專案名稱
TAGS = ["sidewalk", "yolov11", "taiwan"]  # 實驗標籤

# ========== 模型配置 ==========
MODEL_SIZE = "m"  # 可選: n, s, m, l, x
TASK = "segment"  # segment（分割）或 detect（檢測）查一下

# ========== 訓練參數 ==========
EPOCHS = 100
BATCH_SIZE = 16  # T4 建議 8-16，V100 可用 32
IMAGE_SIZE = 640
DEVICE = 0  # 0 表示第一個 GPU
WORKERS = 4

# ========== 優化器設定 ==========   查一下
OPTIMIZER = "AdamW"
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0005

# ========== 數據增強 ==========   MOSAIC查一下
AUGMENT = True
HSV_H = 0.015  # 色調變化
HSV_S = 0.7    # 飽和度變化
HSV_V = 0.4    # 亮度變化
DEGREES = 10.0 # 旋轉角度
TRANSLATE = 0.1 # 平移比例
SCALE = 0.5    # 縮放比例
MOSAIC = 1.0   # Mosaic 增強機率

print(f"📝 實驗名稱: {EXPERIMENT_NAME}")
print(f"🎯 模型: YOLO11_{MODEL_SIZE}_{TASK}")
print(f"⚙️ 訓練配置: {EPOCHS} epochs, batch={BATCH_SIZE}, img={IMAGE_SIZE}")

---
## 📊 第五步：準備數據集配置

YOLO 需要一個 `data.yaml` 檔案來指定數據位置和類別

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ========== 數據集路徑配置 ==========
# 選擇你的數據集來源

# 方法 1: Google Drive
DATASET_PATH = "/content/drive/MyDrive/taiwan_blind_navigation/dataset"

# 方法 2: Colab 本地（如果已下載）
# DATASET_PATH = "/content/dataset"

# 方法 3: GitHub 專案內
# DATASET_PATH = "/content/OpenAIDevice_For_VisualImpairment/dataset"

# ========== 類別定義 ==========
# 根據你的任務修改類別
CLASS_NAMES = [
    "sidewalk_edge",   # 人行道邊緣
    "crosswalk",       # 斑馬線
    "motorcycle",      # 機車
    "pole",            # 電線桿
    "stair"            # 樓梯/台階
]

# ========== 自動生成 data.yaml ==========
data_yaml = {
    'path': DATASET_PATH,
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',  # 可選
    'nc': len(CLASS_NAMES),
    'names': CLASS_NAMES
}

# 儲存 yaml 檔案
yaml_path = '/content/taiwan_data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print(f"✅ 數據配置已生成: {yaml_path}")
print(f"📁 數據路徑: {DATASET_PATH}")
print(f"🏷️ 類別數量: {len(CLASS_NAMES)}")
print(f"📝 類別列表: {CLASS_NAMES}")

---
## 🚀 第六步：開始訓練！

這一步會：
1. 載入預訓練模型
2. 初始化 WandB
3. 開始訓練並自動記錄指標

In [ ]:
from ultralytics import YOLO

# ========== 初始化 WandB ==========
wandb.init(
    project=WANDB_PROJECT,
    name=EXPERIMENT_NAME,
    tags=TAGS,
    config={
        "model_size": MODEL_SIZE,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "image_size": IMAGE_SIZE,
        "optimizer": OPTIMIZER,
        "learning_rate": LEARNING_RATE,
        "augment": AUGMENT,
        "classes": CLASS_NAMES
    }
)

print("✅ WandB 初始化完成！")
print(f"📊 Dashboard: https://wandb.ai/{wandb.run.entity}/{WANDB_PROJECT}/runs/{wandb.run.id}")

In [ ]:
# ========== 載入模型 ==========
model_name = f"yolo11{MODEL_SIZE}-{TASK}.pt"
model = YOLO(model_name)

print(f"✅ 模型載入完成: {model_name}")
print(f"📦 模型參數量: {sum(p.numel() for p in model.model.parameters())/1e6:.2f}M")

In [ ]:
# ========== 開始訓練 ==========
results = model.train(
    data=yaml_path,
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    device=DEVICE,
    workers=WORKERS,

    # 優化器設定
    optimizer=OPTIMIZER,
    lr0=LEARNING_RATE,
    lrf=0.01,
    momentum=0.937,
    weight_decay=WEIGHT_DECAY,

    # 數據增強
    augment=AUGMENT,
    hsv_h=HSV_H,
    hsv_s=HSV_S,
    hsv_v=HSV_V,
    degrees=DEGREES,
    translate=TRANSLATE,
    scale=SCALE,
    mosaic=MOSAIC,

    # WandB 整合
    project=WANDB_PROJECT,
    name=EXPERIMENT_NAME,

    # 其他設定
    save=True,
    save_period=10,  # 每 10 epochs 儲存一次
    plots=True,
    verbose=True
)

print("🎉 訓練完成！")

---
## 📈 第七步：評估模型

In [ ]:
# 在驗證集上評估
metrics = model.val()

# 顯示關鍵指標
print("\n📊 評估結果：")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

if TASK == "segment":
    print(f"\n分割指標：")
    print(f"Mask mAP50: {metrics.seg.map50:.4f}")
    print(f"Mask mAP50-95: {metrics.seg.map:.4f}")

# 記錄到 WandB
wandb.log({
    "final_mAP50": metrics.box.map50,
    "final_mAP50-95": metrics.box.map,
})

if TASK == "segment":
    wandb.log({
        "final_mask_mAP50": metrics.seg.map50,
        "final_mask_mAP50-95": metrics.seg.map,
    })

---
## 🔍 第八步：視覺化預測結果

In [ ]:
# 在測試圖片上預測
test_image = f"{DATASET_PATH}/images/val/sample_001.jpg"  # 修改成你的測試圖片

results = model.predict(
    source=test_image,
    conf=0.25,
    save=True,
    save_txt=True
)

# 顯示結果
from IPython.display import Image, display
display(Image(results[0].path))

# 上傳到 WandB
wandb.log({"prediction_sample": wandb.Image(results[0].path)})

In [ ]:
# 批次預測驗證集（前 10 張）
import glob

val_images = glob.glob(f"{DATASET_PATH}/images/val/*.jpg")[:10]
results = model.predict(source=val_images, conf=0.25, save=True)

# 上傳結果到 WandB
for i, result in enumerate(results):
    wandb.log({f"val_prediction_{i}": wandb.Image(result.path)})

print(f"✅ 已預測 {len(results)} 張驗證圖片")

---
## 💾 第九步：儲存模型

In [ ]:
# 訓練結果會自動儲存在 runs/segment/train/ 或 runs/detect/train/
# 找到最新的訓練結果
import glob
train_dir = sorted(glob.glob('runs/*/train*'))[-1]
best_model = f"{train_dir}/weights/best.pt"
last_model = f"{train_dir}/weights/last.pt"

print(f"📁 訓練結果目錄: {train_dir}")
print(f"🏆 最佳模型: {best_model}")
print(f"📦 最後模型: {last_model}")

# 複製到 Google Drive（如果有掛載）
try:
    output_dir = f"/content/drive/MyDrive/taiwan_blind_navigation/models/{EXPERIMENT_NAME}"
    !mkdir -p {output_dir}
    !cp {best_model} {output_dir}/best.pt
    !cp {last_model} {output_dir}/last.pt
    !cp {train_dir}/results.png {output_dir}/
    !cp {train_dir}/confusion_matrix.png {output_dir}/
    print(f"✅ 模型已複製到 Google Drive: {output_dir}")
except Exception as e:
    print(f"⚠️ 無法複製到 Drive: {e}")

# 下載到本機
from google.colab import files
files.download(best_model)
print("✅ 模型已開始下載到本機！")

In [ ]:
# 上傳模型到 WandB Artifacts（推薦）
artifact = wandb.Artifact(
    name=f"{EXPERIMENT_NAME}_model",
    type="model",
    description=f"Best model from {EXPERIMENT_NAME}",
    metadata={
        "mAP50": float(metrics.box.map50),
        "mAP50-95": float(metrics.box.map),
        "epochs": EPOCHS,
        "classes": CLASS_NAMES
    }
)

artifact.add_file(best_model)
artifact.add_file(last_model)
artifact.add_file(f"{train_dir}/results.png")
wandb.log_artifact(artifact)

print("✅ 模型已上傳到 WandB Artifacts！")

---
## 📤 第十步：導出模型（可選）

In [ ]:
# 導出為 ONNX 格式（用於部署）
model_export = YOLO(best_model)
model_export.export(format="onnx", imgsz=IMAGE_SIZE)

print("✅ 模型已導出為 ONNX 格式！")

In [ ]:
# 導出為 TensorRT 格式（GPU 加速推理）
# model_export.export(format="engine", imgsz=IMAGE_SIZE, device=0)
# print("✅ 模型已導出為 TensorRT 格式！")

---
## 🎯 第十一步：完成訓練

In [ ]:
# 關閉 WandB
wandb.finish()

print("\n" + "="*50)
print("🎉 訓練流程全部完成！")
print("="*50)
print(f"\n📊 WandB Dashboard: https://wandb.ai/{wandb.run.entity}/{WANDB_PROJECT}")
print(f"🏆 最佳模型: {best_model}")
print(f"📈 mAP50: {metrics.box.map50:.4f}")
print(f"📈 mAP50-95: {metrics.box.map:.4f}")
print("\n✨ 記得與組員分享你的實驗結果！")

---
## 🔧 附加工具

### 1. 查看訓練曲線

In [ ]:
from IPython.display import Image, display

# 顯示訓練結果圖
display(Image(f"{train_dir}/results.png"))
display(Image(f"{train_dir}/confusion_matrix.png"))

### 2. 比較不同實驗

In [ ]:
# 前往 WandB Dashboard 查看所有組員的實驗對比
print(f"🔗 比較實驗: https://wandb.ai/your-team/{WANDB_PROJECT}/table")

### 3. 疑難排解

In [ ]:
# 如果遇到 CUDA out of memory，嘗試減少 batch size
# BATCH_SIZE = 8  # 或更小

# 如果訓練太慢，嘗試減少 workers
# WORKERS = 2

# 查看 GPU 使用情況
!nvidia-smi

---
## 📚 資源連結

- [Ultralytics YOLO 文檔](https://docs.ultralytics.com)
- [WandB 教學](https://docs.wandb.ai)
- [專案 GitHub](https://github.com/Djude1/OpenAIDevice_For_VisualImpairment)
- [團隊討論區](https://github.com/Djude1/OpenAIDevice_For_VisualImpairment/discussions)

---
### 👥 組員協作提醒

1. **命名規則**: 使用清晰的 `EXPERIMENT_NAME`
2. **及時推送**: 訓練完記得推送代碼到 GitHub
3. **分享結果**: 在 WandB 標註重要實驗
4. **Code Review**: 互相檢查代碼品質
5. **文檔更新**: 記錄重要發現和改進

🎯 **目標**: 找出最適合台灣環境的模型配置！